In [33]:
%load_ext autoreload
%autoreload 2

from denoised_dataset import HaeaVocab, DenoisingDataset
import torch

vocab = HaeaVocab(torch.zeros(1000, 300, 100), torch.zeros(1000, 300, 100), max_len=20)

print(len(vocab))

dataset = DenoisingDataset(
    vocab,
    127,
    mask=0.3,
    mask_random=0,
    insert=0,
    rotate=0,
    permute_sentences=0,
    replace_length=-1,
    mask_length='span-poisson',
    poisson_lambda=3
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1004


In [29]:
print(dataset.dataset)

tensor([[   1,    4,    5,  ...,   22,   23,    2],
        [   1,    5,    6,  ...,   23,   24,    2],
        [   1,    6,    7,  ...,   24,   25,    2],
        ...,
        [   1,  983,  984,  ..., 1001, 1002,    2],
        [   1,  984,  985,  ..., 1002, 1003,    2],
        [   1,  985,  986,  ..., 1003, 1004,    2]])


In [35]:
print(dataset[0]['source_dataset'].shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [20]:
data = torch.zeros(10, 10)
print(data[0].unsqueeze(0).repeat_interleave(5, dim=0).shape)

torch.Size([5, 10])
